In [1]:
## part 1 dataset
import os
os.environ['http_proxy'] = 'http://proxygate2.ctripcorp.com:8080'
os.environ['https_proxy'] = 'http://proxygate2.ctripcorp.com:8080'
import re
import json
import random
from tool_registry import dispatch_tool, get_tools
tools = get_tools()

def extract_code(text: str) -> str:
    pattern = r'```([^\n]*)\n(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[-1][1]

def tool_call(*args, **kwargs) -> dict:
    print("=== Tool call:")
    print(args)
    print(kwargs)
    return kwargs



with open("/data/share_user/zzd/data/rlhf_data/sft_data_v4/poi_part1.txt", "r") as file:
    part1 = file.read().split('\n')

with open("/data/share_user/zzd/data/rlhf_data/sft_data_v4/poi_part2.txt", "r") as file:
    part2 = file.read().split('\n')

p1_list = random.sample(part1, 300)
q_list = ['{}天气怎么样', '帮我查查{}的天气', '查一下{}的天气', '{}当前的气温如何', '现在{}是晴天是阴天', '请问现在{}的天气如何？', 
          '能否告诉我{}现在的天气情况？', '我想知道{}现在的气温和天气预报', '请问{}今天会下雨吗？']

chatglm_out = "get_weather\n ```python\ntool_call(city_name='{}')\n```"
wendao_out = "get_weather\n ```python\ntool_call(location='{}')\n```"

tmp_list = []
idx = 0
for poi in p1_list:
    question = random.choice(q_list).format(poi)
    chatglm_answer = chatglm_out.format(poi)
    wendao_answer = wendao_out.format(poi)
    round = [{'role':'user', 'content':question}, {'role':'tool', 'content':wendao_answer}]

    tool, *output_text = wendao_answer.strip().split('\n')
    output_text = '\n'.join(output_text)

    code = extract_code(output_text)
    args = eval(code, {'tool_call': tool_call}, {})
    observation = dispatch_tool(tool, args)
    round.append({'role':'observation','content':observation})
    

    norm_d = {'conversations':round,
              'id':idx }
    tmp_list.append(norm_d)

new_file_path = '/data/share_user/zzd/data/rlhf_data/sft_data_v4/weather_data_tmp.json'
with open(new_file_path, "w", encoding="utf-8") as dump_f:
    json.dump(tmp_list, dump_f, ensure_ascii=False)


[registered tool] {'description': 'Get the current weather for `location`',
 'name': 'get_weather',
 'params': [{'description': 'The name of the location to be queried',
             'name': 'location',
             'required': True,
             'type': 'str'}]}
=== Tool call:
()
{'location': '山海关区天下第一关'}
=== Tool call:
()
{'location': '黄山风景区云谷寺'}
=== Tool call:
()
{'location': '甘肃天水'}
=== Tool call:
()
{'location': '云南宾川'}
=== Tool call:
()
{'location': '北京灵光寺'}
=== Tool call:
()
{'location': '长春吉林省博物院'}
=== Tool call:
()
{'location': '洛阳关林镇'}
=== Tool call:
()
{'location': '恩施见天坝瀑布'}
=== Tool call:
()
{'location': '苏州得月楼'}
=== Tool call:
()
{'location': '宜昌交运两坝一峡游船'}
=== Tool call:
()
{'location': '浙江乌镇'}
=== Tool call:
()
{'location': '阜康五江温泉城'}
=== Tool call:
()
{'location': '重庆长寿湖'}
=== Tool call:
()
{'location': '丽江白水河'}
=== Tool call:
()
{'location': '长春般若寺'}
=== Tool call:
()
{'location': '承德热河革命烈士纪念馆'}
=== Tool call:
()
{'location': '阳朔燕莎热气球滑翔伞飞行'}
=== Tool call:
()
{'locatio

In [2]:
import os
import ast
import json
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

actor_path = '/data/share_user/zzd/ckpt/rlhf_baichuan2/1214-f/sft_hf'
def init_model():
    print("init actor model ...")
    model = AutoModelForCausalLM.from_pretrained(
        actor_path,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map='auto'
    ).eval()
    model.generation_config = GenerationConfig.from_pretrained(actor_path)
    tokenizer = AutoTokenizer.from_pretrained(actor_path, use_fast=False, trust_remote_code=True)
    tokenizer.padding_side = 'left'

    return model, tokenizer

model, tokenizer = init_model()


init actor model ...
